In [1]:
import torch
import torchvision.models as models
!pip install onnx onnxruntime
import onnx
import onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [2]:
# Установка пакета из форка репозитория onnx2torch
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Cloning https://github.com/KaizerloveRussia/onnx2torch.git to ./src/onnx2torch
  Running command git clone --filter=blob:none --quiet https://github.com/KaizerloveRussia/onnx2torch.git /content/src/onnx2torch
  Resolved https://github.com/KaizerloveRussia/onnx2torch.git to commit ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_c

In [4]:
# Загрузка и запуск предобученной модели
model = models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()
input_data = torch.randn(1, 3, 224, 224)
output = model(input_data)
print(output)

[{'boxes': tensor([[192.5302,   1.5382, 220.9574,  13.9832],
        [  2.0208,  43.6338,  43.7049,  63.9290],
        [  2.0090,   2.8135,  64.8577,  30.1261],
        [  0.3684,   0.3549,  26.9777,  10.4699],
        [182.0496,   3.1583, 223.0789,  31.0654],
        [  0.6793,   3.4834,  52.3254,  28.0355],
        [  2.2476,  34.9929,  47.7610,  56.5762],
        [ 36.7240, 132.6317,  73.1510, 155.9857],
        [  2.0208,  43.6338,  43.7049,  63.9290],
        [104.0108, 179.5073, 145.3237, 199.8739],
        [  2.3941,  25.8625,  48.7268,  47.8600],
        [177.5819,  87.5226, 221.8432, 111.3307],
        [177.9108,  32.4053, 222.1209,  55.6809],
        [104.4742, 169.8121, 144.0991, 190.6436],
        [  1.0997, 100.3429,  39.4818, 127.9103],
        [  1.7224,  49.7008,  43.0626,  71.6896],
        [167.5007,  66.8278, 212.0858,  89.1039],
        [  2.0090,   2.8135,  64.8577,  30.1261],
        [  2.0208,  43.6338,  43.7049,  63.9290],
        [167.8394,  44.2294, 215.8356, 

In [5]:
# Конвертация модели в ONNX
torch.onnx.export(model, input_data, "retinanet_resnet50_fpn.onnx", opset_version=13)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4009: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bo

In [6]:
# Загрузка и запуск модели в ONNX
onnx_model = onnx.load("retinanet_resnet50_fpn.onnx")
ort_session = onnxruntime.InferenceSession("retinanet_resnet50_fpn.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: input_data.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

Результат запуска модели RetinaNet в ONNX: [array([[192.53017   ,   1.5382231 , 220.95738   ,  13.983224  ],
       [  2.020795  ,  43.633785  ,  43.7049    ,  63.92896   ],
       [  2.0090008 ,   2.8135533 ,  64.857735  ,  30.126114  ],
       [  0.3684175 ,   0.3549459 ,  26.977686  ,  10.469933  ],
       [182.04953   ,   3.1582448 , 223.07889   ,  31.06547   ],
       [  0.67931277,   3.4834101 ,  52.325375  ,  28.035475  ],
       [  2.2476478 ,  34.992855  ,  47.76096   ,  56.576225  ],
       [ 36.724022  , 132.6317    ,  73.15099   , 155.98564   ],
       [  2.020795  ,  43.633785  ,  43.7049    ,  63.92896   ],
       [104.01076   , 179.50728   , 145.32375   , 199.87386   ],
       [  2.3940928 ,  25.8625    ,  48.726818  ,  47.859978  ],
       [177.58182   ,  87.52263   , 221.84325   , 111.33073   ],
       [177.91074   ,  32.405334  , 222.12094   ,  55.68088   ],
       [104.47416   , 169.81204   , 144.09917   , 190.64363   ],
       [  1.0996972 , 100.342896  ,  39.481785

In [7]:
# Обратная конвертация
from onnx2torch.converter import convert
torch_model = convert("retinanet_resnet50_fpn.onnx")

ModuleNotFoundError: No module named 'onnx2torch'